In [3]:
import pickle
from sklearn.metrics import roc_auc_score, log_loss
project = 'openstack'
is_its = 'in_its'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data = pickle.load(f_train)
        te_data = pickle.load(f_test)
pred_list = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+is_its+'/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list.append(pickle.load(f))
tr_pred = [[pred_list[0][j], pred_list[1][j], pred_list[2][j]] for j in range(len(pred_list[0]))]
te_pred = [[pred_list[3][j], pred_list[4][j], pred_list[5][j]] for j in range(len(pred_list[3]))]                
tr_data = [tr_pred, tr_data[5]]
te_data = [te_pred, te_data[5]]

In [4]:
import pickle
from sklearn.metrics import roc_auc_score, log_loss
project = 'openstack'
is_its = 'no_its'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data2 = pickle.load(f_train)
        te_data2 = pickle.load(f_test)
pred_list2 = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+is_its+'/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list2.append(pickle.load(f))
tr_pred = [[pred_list2[0][j], pred_list2[1][j], pred_list2[2][j]] for j in range(len(pred_list2[0]))]
te_pred = [[pred_list2[3][j], pred_list2[4][j], pred_list2[5][j]] for j in range(len(pred_list2[3]))]                
tr_data2 = [tr_pred, tr_data2[5]]
te_data2 = [te_pred, te_data2[5]]

In [5]:
import pickle
from sklearn.metrics import roc_auc_score, log_loss
project = 'qt'
is_its = 'in_its'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data3 = pickle.load(f_train)
        te_data3 = pickle.load(f_test)
pred_list3 = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+is_its+'/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list3.append(pickle.load(f))
tr_pred = [[pred_list3[0][j], pred_list3[1][j], pred_list3[2][j]] for j in range(len(pred_list3[0]))]
te_pred = [[pred_list3[3][j], pred_list3[4][j], pred_list3[5][j]] for j in range(len(pred_list3[3]))]                
tr_data3 = [tr_pred, tr_data3[5]]
te_data3 = [te_pred, te_data3[5]]

In [6]:
import pickle
from sklearn.metrics import roc_auc_score, log_loss
project = 'qt'
is_its = 'no_its'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data4 = pickle.load(f_train)
        te_data4 = pickle.load(f_test)
pred_list4 = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+is_its+'/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list4.append(pickle.load(f))
tr_pred = [[pred_list4[0][j], pred_list4[1][j], pred_list4[2][j]] for j in range(len(pred_list4[0]))]
te_pred = [[pred_list4[3][j], pred_list4[4][j], pred_list4[5][j]] for j in range(len(pred_list4[3]))]                
tr_data4 = [tr_pred, tr_data4[5]]
te_data4 = [te_pred, te_data4[5]]

In [112]:
lgb_weight = 0.313
code_weight = 0.436
msg_weight = 0.251

In [73]:
#openstack valid
# #code + metrics
# lgb_weight2 = lgb_weight / sum([lgb_weight, code_weight])
# code_weight2 = code_weight / sum([lgb_weight, code_weight])
# weighted_average = lgb_weight2 * pred_list[0] + code_weight2 * pred_list[1]
# print(f'test auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(tr_data[1], weighted_average):.3f}')
# #in_its_msg + metrics
# lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
# msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
# weighted_average = lgb_weight2 * pred_list[0] + msg_weight2 * pred_list[2]
# print(f'test auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(tr_data[1], weighted_average):.3f}')
#in_its_msg + code
code_weight2 = code_weight / sum([code_weight, msg_weight])
msg_weight2 = msg_weight / sum([code_weight, msg_weight])
weighted_average = code_weight2 * pred_list[1] + msg_weight2 * pred_list[2]
print(f'test auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data[1], weighted_average):.3f}')
#ensemble(in_its_msg)
weighted_average = lgb_weight * pred_list[0] + code_weight * pred_list[1] + msg_weight * pred_list[2]
print(f'test auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data[1], weighted_average):.3f}')
# #no_its_msg + metrics
# lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
# msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
# weighted_average = lgb_weight2 * pred_list[0] + msg_weight2 * pred_list2[2]
# print(f'test auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(tr_data[1], weighted_average):.3f}')
# #no_its_msg + code
# code_weight2 = code_weight / sum([code_weight, msg_weight])
# msg_weight2 = msg_weight / sum([code_weight, msg_weight])
# weighted_average = code_weight2 * pred_list[1] + msg_weight2 * pred_list2[2]
# print(f'test auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(tr_data[1], weighted_average):.3f}')
# #ensemble(no_its_msg)
# weighted_average = lgb_weight * pred_list[0] + code_weight * pred_list[1] + msg_weight * pred_list2[2]
# print(f'test auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(tr_data[1], weighted_average):.3f}')

test auc:0.803
test loss:0.430
test auc:0.814
test loss:0.479


In [111]:
#openstack test
# #code + metrics
# lgb_weight2 = lgb_weight / sum([lgb_weight, code_weight])
# code_weight2 = code_weight / sum([lgb_weight, code_weight])
# weighted_average = lgb_weight2 * pred_list[3] + code_weight2 * pred_list[4]
# print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(te_data[1], weighted_average):.3f}')
# #in_its_msg + metrics
# lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
# msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
# weighted_average = lgb_weight2 * pred_list[3] + msg_weight2 * pred_list[5]
# print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(te_data[1], weighted_average):.3f}')
#in_its_msg + code
code_weight2 = code_weight / sum([code_weight, msg_weight])
msg_weight2 = msg_weight / sum([code_weight, msg_weight])
weighted_average = code_weight2 * pred_list[4] + msg_weight2 * pred_list[5]
print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data[1], weighted_average):.3f}')
#ensemble(in_its_msg)
weighted_average = lgb_weight * pred_list[3] + code_weight * pred_list[4] + msg_weight * pred_list[5]
print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data[1], weighted_average):.3f}')
# #no_its_msg + metrics
# lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
# msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
# weighted_average = lgb_weight2 * pred_list[3] + msg_weight2 * pred_list2[5]
# print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average):.3f}')
# print(f'test loss:{log_loss(te_data[1], weighted_average):.3f}')
#no_its_msg + code
code_weight2 = code_weight / sum([code_weight, msg_weight])
msg_weight2 = msg_weight / sum([code_weight, msg_weight])
weighted_average = code_weight2 * pred_list[4] + msg_weight2 * pred_list2[5]
print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data[1], weighted_average):.3f}')
#ensemble(no_its_msg)
weighted_average = lgb_weight * pred_list[3] + code_weight * pred_list[4] + msg_weight * pred_list2[5]
print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data[1], weighted_average):.3f}')

test auc:0.803
test loss:0.521
test auc:0.823
test loss:0.537
test auc:0.803
test loss:0.545
test auc:0.823
test loss:0.548


In [103]:
#qt valid
#code + metrics
lgb_weight2 = lgb_weight / sum([lgb_weight, code_weight])
code_weight2 = code_weight / sum([lgb_weight, code_weight])
weighted_average = lgb_weight2 * pred_list3[0] + code_weight2 * pred_list3[1]
print(f'test auc:{roc_auc_score(tr_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data3[1], weighted_average):.3f}')
#in_its_msg + metrics
lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
weighted_average = lgb_weight2 * pred_list3[0] + msg_weight2 * pred_list3[2]
print(f'test auc:{roc_auc_score(tr_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data3[1], weighted_average):.3f}')
#in_its_msg + code
code_weight2 = code_weight / sum([code_weight, msg_weight])
msg_weight2 = msg_weight / sum([code_weight, msg_weight])
weighted_average = code_weight2 * pred_list3[1] + msg_weight2 * pred_list3[2]
print(f'test auc:{roc_auc_score(tr_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data3[1], weighted_average):.3f}')
#ensemble(in_its_msg)
weighted_average = lgb_weight * pred_list3[0] + code_weight * pred_list3[1] + msg_weight * pred_list3[2]
print(f'test auc:{roc_auc_score(tr_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data3[1], weighted_average):.3f}')
#no_its_msg + metrics
lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
weighted_average = lgb_weight2 * pred_list3[0] + msg_weight2 * pred_list4[2]
print(f'test auc:{roc_auc_score(tr_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data3[1], weighted_average):.3f}')
#no_its_msg + code
code_weight2 = code_weight / sum([code_weight, msg_weight])
msg_weight2 = msg_weight / sum([code_weight, msg_weight])
weighted_average = code_weight2 * pred_list3[1] + msg_weight2 * pred_list4[2]
print(f'test auc:{roc_auc_score(tr_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data3[1], weighted_average):.3f}')
#ensemble(no_its_msg)
weighted_average = lgb_weight * pred_list3[0] + code_weight * pred_list3[1] + msg_weight * pred_list4[2]
print(f'test auc:{roc_auc_score(tr_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(tr_data3[1], weighted_average):.3f}')

test auc:0.812
test loss:0.505
test auc:0.785
test loss:0.462
test auc:0.804
test loss:0.502
test auc:0.815
test loss:0.484
test auc:0.783
test loss:0.444
test auc:0.804
test loss:0.483
test auc:0.815
test loss:0.473


In [113]:
#qt
#code + metrics
lgb_weight2 = lgb_weight / sum([lgb_weight, code_weight])
code_weight2 = code_weight / sum([lgb_weight, code_weight])
weighted_average = lgb_weight2 * pred_list3[3] + code_weight2 * pred_list3[4]
print(f'test auc:{roc_auc_score(te_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data3[1], weighted_average):.3f}')

#in_its_msg + metrics
lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
weighted_average = lgb_weight2 * pred_list3[3] + msg_weight2 * pred_list3[5]
print(f'test auc:{roc_auc_score(te_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data3[1], weighted_average):.3f}')

#in_its_msg + code
code_weight2 = code_weight / sum([code_weight, msg_weight])
msg_weight2 = msg_weight / sum([code_weight, msg_weight])
weighted_average = code_weight2 * pred_list3[4] + msg_weight2 * pred_list3[5]
print(f'test auc:{roc_auc_score(te_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data3[1], weighted_average):.3f}')

#ensemble(in_its_msg)
weighted_average = lgb_weight * pred_list3[3] + code_weight * pred_list3[4] + msg_weight * pred_list3[5]
print(f'test auc:{roc_auc_score(te_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data3[1], weighted_average):.3f}')

#no_its_msg + metrics
lgb_weight2 = lgb_weight / sum([lgb_weight, msg_weight])
msg_weight2 = msg_weight / sum([lgb_weight, msg_weight])
weighted_average = lgb_weight2 * pred_list3[3] + msg_weight2 * pred_list4[5]
print(f'test auc:{roc_auc_score(te_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data3[1], weighted_average):.3f}')

#no_its_msg + code
code_weight2 = code_weight / sum([code_weight, msg_weight])
msg_weight2 = msg_weight / sum([code_weight, msg_weight])
weighted_average = code_weight2 * pred_list3[4] + msg_weight2 * pred_list4[5]
print(f'test auc:{roc_auc_score(te_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data3[1], weighted_average):.3f}')

#ensemble(no_its_msg)
weighted_average = lgb_weight * pred_list3[3] + code_weight * pred_list3[4] + msg_weight * pred_list4[5]
print(f'test auc:{roc_auc_score(te_data3[1] ,weighted_average):.3f}')
print(f'test loss:{log_loss(te_data3[1], weighted_average):.3f}')

test auc:0.790
test loss:0.534
test auc:0.769
test loss:0.397
test auc:0.729
test loss:0.497
test auc:0.775
test loss:0.472
test auc:0.790
test loss:0.353
test auc:0.740
test loss:0.454
test auc:0.783
test loss:0.443


In [12]:
print(project, is_its)
for weight in weights:
    weighted_average = weight[0] * pred_list[0] + weight[1] * pred_list[1] + weight[2] * pred_list[2]
    weighted_average2 = weight[0] * pred_list[3] + weight[1] * pred_list[4] + weight[2] * pred_list[5]
    print(weight)
    print(f'train auc:{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
    print(f'train loss:{log_loss(tr_data[1], weighted_average):.3f}')
    print(f'test auc:{roc_auc_score(te_data[1] ,weighted_average2):.3f}')
    print(f'test loss:{log_loss(te_data[1], weighted_average2):.3f}')

qt no_its


NameError: name 'weights' is not defined

In [ ]:
weighted_average = 0.4 * pred_list[0] + 0.45 * pred_list[1] + 0.15 * pred_list[2]
print(f'{roc_auc_score(tr_data[1] ,weighted_average):.3f}')
print(f'{log_loss(tr_data[1], weighted_average):.3f}')


0.815
0.462


In [ ]:
weighted_average2 = 0.4 * pred_list[3] + 0.45 * pred_list[4] + 0.15 * pred_list[5]
print(f'{roc_auc_score(te_data[1] ,weighted_average2):.3f}')
print(f'{log_loss(te_data[1], weighted_average2):.3f}')

0.821
0.534


In [55]:
#Qt std
import numpy as np
std = np.array([1/0.008771, 1/0.010731, 1/0.009309])
norm_std = std / np.sum(std)
print(f'{norm_std[0]:.3f} {norm_std[1]:.3f} {norm_std[2]:.3f}')

std2 = np.array([1/0.020657, 1/0.030968, 1/0.060467])
norm_std2 = std2 / np.sum(std2)
print(f'{norm_std2[0]:.3f} {norm_std2[1]:.3f} {norm_std2[2]:.3f}')

0.362 0.296 0.341
0.498 0.332 0.170


In [56]:
# OpenStack std
import numpy as np
std = np.array([1/0.008330, 1/0.012456, 1/0.015932])
norm_std = std / np.sum(std)
print(f'{norm_std[0]:.3f} {norm_std[1]:.3f} {norm_std[2]:.3f}')

std2 = np.array([1/0.006839, 1/0.064984, 1/0.034017])
norm_std2 = std2 / np.sum(std2)
print(f'{norm_std2[0]:.3f} {norm_std2[1]:.3f} {norm_std2[2]:.3f}')

0.456 0.305 0.239
0.766 0.081 0.154


In [89]:
#Qt weighted_ratio
import numpy as np
from scipy import stats
metrics_weight = [0.425, 0.383,0.533]
code_weight = [0.486, 0.313, 0.356]
msg_weight = [0.089, 0.305, 0.111]
weights = [np.mean(metrics_weight), np.mean(code_weight), np.mean(msg_weight)]
# weights = [stats.hmean(metrics_weight), stats.hmean(code_weight), stats.hmean(msg_weight)]
print(weights)
normalize = weights / sum(weights)
print(f'metrics:{normalize[0]:.3f},code:{normalize[1]:.3f},msg:{normalize[2]:.3f}')

[0.44700000000000006, 0.38499999999999995, 0.16833333333333333]
metrics:0.447,code:0.385,msg:0.168


In [88]:
#OpenStack weighted_ratio
import numpy as np
metrics_weight = [0.500, 0.456, 0.766]
code_weight = [0.358,0.305,0.081]
msg_weight = [0.142,0.239,0.154]
weights = [np.mean(metrics_weight), np.mean(code_weight), np.mean(msg_weight)]
# weights = [stats.hmean(metrics_weight), stats.hmean(code_weight), stats.hmean(msg_weight)]
print(weights)
normalize = weights / sum(weights)
print(f'metrics:{normalize[0]:.3f},code:{normalize[1]:.3f},msg:{normalize[2]:.3f}')

[0.574, 0.248, 0.17833333333333334]
metrics:0.574,code:0.248,msg:0.178
